# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam

from scipy.stats import percentileofscore
import time
import math

# Training data

In [ ]:
df = pd.read_parquet('../pipeline/tmp/cities/poz-w/dataset.parquet')
df.head(5)

In [ ]:
print(df.size)

In [ ]:
x = df['to_x']
y = df['to_y']

plt.figure(figsize=(6, 5))
plt.hist2d(x, y, bins=100, cmap='viridis')

plt.colorbar(label='Counts')

In [ ]:
X = df[['from_x', 'from_y', 'to_x', 'to_y', 'day_type', 'start', 'reference']]
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Model definition

## architecture

In [ ]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.body = Sequential([
            Dense(64, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(2),
        ])

    def call(self, inputs):
        reference = inputs[:, 6]
        mods = self.body(inputs[:, :6])
        return reference * (1 + mods[:, 0]) + 15 * 60 * mods[:, 1]

## loss function

In [ ]:
def asymetric_loss_function(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    tf.debugging.assert_greater(
        tf.reduce_min(y_true), tf.constant(0.0, dtype=tf.float32),
        message="y_true contains zero or negative values!"
    )

    e = y_pred - y_true
    se = e**2
    loss = tf.where(e < 0, se, 64*se)
    return tf.reduce_mean(loss)

## compile

In [ ]:
model = Model()
model.compile(optimizer='adam', loss=tf.keras.losses.mse, metrics=['mape'])

# Training

In [ ]:
history = model.fit(X_train, y_train, epochs=16, batch_size=64, validation_split=0.2)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
]
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Model evaluation

## Read tflite model

In [ ]:
with open('model.tflite', 'rb') as f:
    tflite_model = f.read()

interpreter = tf.lite.Interpreter(model_content=tflite_model)

in_idx = interpreter.get_input_details()[0]['index']
out_idx = interpreter.get_output_details()[0]['index']

interpreter.resize_tensor_input(in_idx, (1, 7))
interpreter.allocate_tensors()

## predicitons on test set

In [ ]:
y_pred = []

for i in range(X_test.shape[0]):
    single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float32)
    interpreter.set_tensor(in_idx, single_row)
    interpreter.invoke()
    output_data = interpreter.get_tensor(out_idx)
    y_pred.append(output_data[0])

y_pred = np.array(y_pred)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

## Visualize set results distribution

In [ ]:
percentage_difference = ((y_pred - y_test) / y_test) * 100

percentile_1 = np.percentile(percentage_difference, 1)
percentile_99 = np.percentile(percentage_difference, 99.9)
max_overestimation = np.max(percentage_difference)
percentile_of_0 = percentileofscore(percentage_difference, 0)

plt.figure(figsize=(10, 6))
sns.histplot(percentage_difference, kde=True, bins=100, color='blue')

plt.axvline(percentile_1, color='green', linestyle='--', linewidth=2, label=f'1st Percentile: {percentile_1:.2f}%')
plt.axvline(percentile_99, color='red', linestyle='--', linewidth=2, label=f'99.9th Percentile: {percentile_99:.2f}%')

plt.text(percentile_1, plt.ylim()[1] * 0.9, f'{percentile_1:.2f}%', color='green', ha='center')
plt.text(percentile_99, plt.ylim()[1] * 0.9, f'{percentile_99:.2f}%', color='red', ha='center')

plt.axvline(max_overestimation, color='purple', linestyle='--', linewidth=2, label=f'Max Overestimation: {max_overestimation:.2f}%')
plt.text(max_overestimation, plt.ylim()[1] * 0.8, f'{max_overestimation:.2f}%', color='purple', ha='center')

plt.xlabel('Percentage Difference (%)')
plt.ylabel('Frequency')
plt.title('Distribution of Percentage Difference Between Predictions and Targets')
plt.legend()
plt.show()

print(f'Maximal Overestimation Value: {max_overestimation:.2f}%')
print(f'Percentile for 0 on x-axis value: {percentile_of_0:.2f}%')

## Forward pass time

In [ ]:
num_iterations = 10000
total_time = 0

for _ in range(num_iterations):
    random_index = np.random.randint(0, X_test.shape[0])
    # single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float64)
    single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float32)
    start_time = time.perf_counter()
    interpreter.set_tensor(in_idx, single_row)
    interpreter.invoke()

    output_data = interpreter.get_tensor(out_idx)
    end_time = time.perf_counter()
    total_time += (end_time - start_time)

average_forward_pass_time = (total_time / num_iterations) * 1e6
print(f"Average forward pass time for a single row using TensorFlow Lite: {average_forward_pass_time:.6f} microseconds")
print(f"total time for 10000 rows: {total_time*1000} milliseconds")